In [1]:
# Getting import problems. 
# Trying fix from here: https://github.com/conda-forge/pygridgen-feedstock/issues/10#issuecomment-365914605
# !apt update
# !apt install libgl1-mesa-glx

In [2]:
# only needed 
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager ipygany

## The working example

In [3]:
# install some stuff

# !mamba install -y ipygany pyvista 

# k3d actually messes up my other examples...

# Need to rebuild lab and refresh browser after

In [4]:
from skimage import measure

In [5]:
import matplotlib.pyplot as plt

In [6]:
from cmip6_preprocessing.preprocessing import combined_preprocessing

In [7]:
import numpy as np
from ipywidgets import FloatSlider, FloatRangeSlider, Dropdown, Select, VBox, AppLayout, jslink
from ipygany import Scene, IsoColor, PolyMesh, Component, ColorBar, colormaps, Alpha, IsoSurface

## Loading some bathymetry data

In [8]:
import intake
# newer version
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
col

,unique
activity_id,17
institution_id,36
source_id,86
experiment_id,168
member_id,626
table_id,37
variable_id,709
grid_label,10
zstore,483600
dcpp_init_year,60


In [9]:
aspect_ratio = {'x':1,'y':1,'lev':1/5}
spacing = {'x':1, 'y':1, 'lev':20}

In [10]:
def convert_mol_m3_mymol_kg(o2):
    converted = o2 / 1025 * 1e6
    converted.attrs["units"] = "$\mu mol/kg$"
    return converted

In [11]:
dd_kwargs = dict(zarr_kwargs={'consolidated':True}, preprocess=combined_preprocessing)

In [12]:
cat = col.search(variable_id=['deptho'], grid_label='gr', source_id='GFDL-CM4')
ddict = cat.to_dataset_dict(**dd_kwargs)
_, ds = ddict.popitem()
depth = ds.deptho.fillna(0).squeeze().transpose('y','x').load().data


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


### Get o2 data

In [13]:
cat = col.search(variable_id=['o2'], grid_label='gr', source_id='GFDL-CM4')
ddict = cat.to_dataset_dict(**dd_kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [14]:
ds = ddict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gr']
da_o2 = convert_mol_m3_mymol_kg(ds.o2).squeeze().isel(time=slice(0,600)).mean('time').transpose('x','y','lev')
# For now I have loaded the coarser depth data, need to find a way to harmonize this with the cooler high res data later

# the current marching_cube implementation needs regularly spaced data
da_o2 = da_o2.interp(lev=np.arange(0,np.max(depth),spacing['lev']))

print(da_o2.nbytes/1e9)
o2 = da_o2.fillna(1e12).load().data
# o2

0.0834624


/Users/juliusbusecke/miniconda/envs/hack_3d_viz/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [15]:
def isovolume(data, levels, spacing, colors,  vmin=0, vmax=120):
    if not isinstance(levels, list):
        levels = list(levels)
    scenes = []
    
    for level, color in zip(levels, colors):
        # Use marching cubes to obtain the surface mesh of these ellipsoids
        verts, faces, normals, values = measure.marching_cubes(o2, level, spacing=(1,1,-spacing['lev']))

        # normalize
        verts = verts / np.array(o2.shape) * [aspect_ratio[dim]/spacing[dim] for dim in list(da_o2.dims) if dim in aspect_ratio.keys()]
        
        height_component = Component(name='value', array=o2) # I have no clue what this does?

        mesh = PolyMesh(
            vertices=verts,
            triangle_indices=faces,
            data={'height': [height_component]}
        )
        
        margin = 3 # this is a really shitty solution. I want a solid color for the isosurface not a value mapped onto it (maybe later)
        height_min = level - margin
        height_max = level + margin
        
        colored_mesh = Alpha(mesh, alpha=0.1)
        colored_mesh.default_color = color
    
        # this eliminates the shading....
#         colored_mesh = Alpha(IsoSurface(mesh,rgb='r')) # this is a total guess. The docstrings are not very helpful to figure out kwargs...neither are the docs at this point.
        
        scenes.append(colored_mesh)
    return scenes

In [16]:
iso = isovolume(o2, [20, 80, 140], spacing, ['red', 'blue', 'green'])
AppLayout(center=Scene(iso))

AppLayout(children=(Scene(children=[Alpha(default_color='red', input=0.7, parent=PolyMesh(data=[Data(component…

In [17]:
iso[0].default_color = 'red'

In [18]:
iso[0].class_trait_names()

['_display_callbacks',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_property_lock',
 '_states_to_send',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'comm',
 'default_color',
 'environment_meshes',
 'input',
 'keys',
 'log',
 'parent',
 'vertices']

Some notes

- transpose matters (y,x order)
- z (and all other dims) needs to be normalized. Large depths messed up the figure!

In [19]:
# Create triangle indices


ny, nx = depth.shape
triangle_indices = np.empty((ny - 1, nx - 1, 2, 3), dtype=int)

r = np.arange(nx * ny).reshape(ny, nx)

triangle_indices[:, :, 0, 0] = r[:-1, :-1]
triangle_indices[:, :, 1, 0] = r[:-1, 1:]
triangle_indices[:, :, 0, 1] = r[:-1, 1:]

triangle_indices[:, :, 1, 1] = r[1:, 1:]
triangle_indices[:, :, :, 2] = r[1:, :-1, None]

triangle_indices.shape = (-1, 3)

# Create vertices
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)

xx, yy = np.meshgrid(x, y, sparse=True)

z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)
# z = np.random.rand(ny,nx)
z = abs(depth)
z_norm = z/np.max(abs(z))

vertices = np.empty((ny, nx, 3))
vertices[:, :, 0] = xx * aspect_ratio['x']
vertices[:, :, 1] = yy * aspect_ratio['y']
vertices[:, :, 2] = -z_norm * aspect_ratio['lev']
vertices = vertices.reshape(nx * ny, 3)

height_component = Component(name='value', array=z)

bathymetry = PolyMesh(
    vertices=vertices,
    triangle_indices=triangle_indices,
    data={'height': [height_component]}
)
bathymetry.default_color = 'gray'

height_min = np.min(z)
height_max = np.max(z)

iso = isovolume(o2, [20, 80], spacing, ['red', 'purple', 'blue'])


AppLayout(
    center=Scene([bathymetry]+iso),
    pane_widths=[0, 3, 0]
)

<ipython-input-19-6de2f4a3a776>:24: RuntimeWarning: invalid value encountered in true_divide
  z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)


AppLayout(children=(Scene(children=[PolyMesh(data=[Data(components=[Component(array=array([[   0.    ,    0.  …